[과제] https://news.naver.com 사이트에서 4개의 뉴스 카테고리를 선택해서 뉴스기사를 크롤링한 후 다음을 수행하세요
- 뉴스기사 카테고리별 저장(파일 or db)
- 텍스트 전처리 및 피처 벡터화
- 모델링 및 평가(성능개선 포함)

Reference  
https://celltong.tistory.com/entry/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%ED%81%AC%EB%A1%A4%EB%A7%81-%EC%8B%A4%EC%8A%B5-%EB%84%A4%EC%9D%B4%EB%B2%84-%EB%89%B4%EC%8A%A4-%EC%84%B9%EC%85%98%EB%93%A4-%EA%B8%B0%EC%82%AC-%EB%82%B4%EC%9A%A9-%EC%B6%94%EC%B6%9C%ED%95%98%EA%B8%B0  
(흐름만 보고 제 상황에 맞게 한거라 조금 다를 수 있습니다.)

https://news.naver.com/main/list.naver?mode=LSD&mid=sec&sid1=001  
위 링크는 네이버뉴스의 언론사별 속보 탭입니다. 저는 추후 도움이 될 것 같아 이것에서 진행하였습니다.  
찾는방법은 네이버뉴스 - 중간 우상단쯤 "전체 언론사" - 하단의 전체 탭에 "언론사 최신기사" 입니다.

In [2]:
# 제 컴퓨터 기준으로 11분정도 소요되었습니다. 참고바랍니다.

from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np

# 단계별 작업을 사용자 함수로 만들어 제작하였음.
def get_request(section=None,page=None):
    global header
    header = {
        'referer': 'https://www.naver.com',
        'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36'
    }
    url = 'https://news.naver.com/main/list.naver'
    # 가서 카테고리별로 보다보면 sid1 뒷부분 숫자가 바뀜
    sections = {
        '정치': 100,
        '경제': 101,
        '사회': 102,
        '생활/문화': 103
    }

    req = requests.get(url,headers=header,params={'sid1':sections[section],'page':page})

    # req = requests.get(url,headers=header,params={'date':20220319,'sid1':sections[section],'page':page})

    # 연습용 데이터 만들 떄 22년 03월 19일 21시에 만들었는데,
    # 코드 실행 당시 뉴스의 페이지가 30페이지까지 없는 경우에는 오류가 날 수 있다. 위와 같이 date를 주게 되면
    # 제작 당시 사용된 날짜로 실행 가능하다. (제작 일시: 22년 03월 19일 21시)

    return req

def get_href(soup=None):
    hrefs = []
    # 기사들 있는 태그 찾기
    div = soup.find('div',class_='list_body newsflash_body')
    for dt in div.find_all('dt'):
        hrefs.append(dt.find('a')['href'])
    return set(hrefs) # 해보면 사진이 있는 경우에는 두번씩 반복됨.. 중복 제거가 필요함

def get_contents(soup=None):
    # 기사 본문이 있는 div를 찾는 과정에서 오류가 나는 경우가 있음. 이 경우는 아마도
    # 기사에 이미지만 있고 내용이 없는 경우가 있어서로 추정됨.
    # 그 예외를 처리해주기 위해 try except 사용하여 건너뛰기 함.
    try:
        div = soup.find('div',id='articleBodyContents')
        # 개행문자가 좀 있길래 미리 처리해주고 뽑았음.
        contents = div.get_text().replace('\n',' ').replace('\t',' ').replace('\xa0',' ')
        return contents
    except AttributeError:
        pass

def crawling(section=None):
    list_href = []
    result = []
    # 우선 링크들을 뽑아온다.
    # 페이지를 끝까지 하도록 하는 방법도 가능하겠으나(div중에 class=paging인 것이 있음), 모델 이해를 위한 연습용 데이터로써 사용될 것이라
    # 섹션별 데이터 개수를 맞춰주기 위해서, 또 혹시나 네이버 서버 트래픽 문제가 발생하지 않도록 적당량의 수만 크롤링 하도록 한다.
    for i in range(1,31):
        req = get_request(section,i)
        soup = bs(req.text,'html.parser')
        list_href += list(get_href(soup))
    # 뽑아온 링크 리스트에 접속하여 기사를 뽑아서 넣는다
    for href in list_href:
        href_req = requests.get(href,headers=header)
        href_soup = bs(href_req.text,'html.parser')
        result.append(get_contents(href_soup))
    return(result)


# 실제 데이터셋을 만드는 코드입니다.  
# 각 섹션별로 기사를 크롤링해오고, 섹션의 이름은 0~3으로 주었습니다.
# (0: 정치, 1: 경제, 2: 사회, 3: 생활/문화)
politics = crawling('정치')
politics = pd.DataFrame(politics,columns=['data'])
politics['section'] = np.full_like(politics.data,0)

economy = crawling('경제')
economy = pd.DataFrame(economy,columns=['data'])
economy['section'] = np.full_like(economy.data,1)

society = crawling('사회')
society = pd.DataFrame(society,columns=['data'])
society['section'] = np.full_like(society.data,2)

culture = crawling('생활/문화')
culture = pd.DataFrame(culture,columns=['data'])
culture['section'] = np.full_like(culture.data,3)



# 크롤링한 데이터프레임들을 index방향으로 합쳐서 하나의 데이터셋으로 만들고 csv파일로 저장합니다.
newsdata = pd.concat([politics,economy,society,culture],axis=0)
newsdata.to_csv('./dataset/newsdata.csv')

In [111]:
# 예시로 보면서 텍스트 전처리의 힌트를 얻으려고 합니다.
economy.iloc[16,0]

"    동영상 뉴스        [앵커]차기 윤석열 정부에서 '통상' 업무를 어느 부처에 둬야 할지가 도마에 올랐습니다.9년 만에 통상 권한을 되찾아 오려는 외교부와 지키려는 산업부 사이의 물밑 신경전도 치열합니다.강정규 기자입니다.[기자]대외 무역을 관장하는 '통상' 기능을 어디에 둘 것인가?논란의 시작은 안철수 대통령직 인수위원장의 후보 시절 공약으로 거슬러 올라갑니다.[안철수 / 당시 국민의당 후보 (지난해 12월) : 산업 통상 자원부의 통상 업무는 외교부로 이관해서 / 과학기술경제외교 시대에 효율적으로 대응하도록 하겠습니다.]통상 권한 이관 문제는 정권 교체 때마다 정부 조직 개편안의 단골 소재였습니다.1998년 김대중 정부 출범과 함께 '통상'이란 두 글자는 산업부에서 외교부로 넘어갔습니다.박근혜 정부 들어 산업부의 후신인 지식경제부로 되돌려 놓으며 갈등을 빚기도 했습니다.[김성환 / 당시 외교통상부 장관 (지난 2013년) : 우리 헌법과 정부조직법의 골간을 흔드는 결과가 초래돼서 대외관계에서 안정성과 일관성을 기할 수 없을 것으로 우려됩니다.]인수위가 10년 만에 부활한 만큼 두 부처 간 물밑 다툼도 치열합니다.외교부에선 CPTPP나 쿼드 가입 등 굵직한 현안의 사령탑을 자신들이 맡아야 한다고 주장합니다.반면 산업부는 국내 산업 기반에 대한 고려 없이 국제 정치 논리로 통상을 다룰 순 없다고 맞섭니다.한미 FTA 10주년 공치사 과정에서 '아전인수'식 행사를 앞다퉈 열기도 했습니다.[김상모 / 산업부 대변인 (지난 7일) : 한미 FTA 10주년 기념 세미나를 개최합니다. 통상협력의 새로운 도약을 모색하는 내용으로….][최영삼 / 외교부 대변인 (지난 15일) : 외교통상부 시절 타결되었던 역사적 한미 FTA 협정 발효 10주년을 맞이해…]미국의 러시아 수출 통제 대상(FDPR)에서 한국이 처음에 면제받지 못한 것도 같은 맥락으로 풀이됩니다.두 부처가 통상 주도권을 두고 엇박자를 내다가 뒤늦게 수습에 나서는 촌극을 빚었다는 겁니다.[여한구 / 

In [43]:
# 밑에 전처리 함수로 처리한 뒤 어떻게 바뀌었는지 확인하고자 삽입한 셀입니다.
# newsdata.iloc[611,0]

'동영상 뉴스 차기 윤석열 정부에서 통상 업무를 어느 부처에 둬야 할지가 도마에 올랐습니다 9년 만에 통상 권한을 되찾아 오려는 외교부와 지키려는 산업부 사이의 물밑 신경전도 치열합니다 강정규 기자입니다 대외 무역을 관장하는 통상 기능을 어디에 둘 것인가 논란의 시작은 안철수 대통령직 인수위원장의 후보 시절 공약으로 거슬러 올라갑니다 통상 권한 이관 문제는 정권 교체 때마다 정부 조직 개편안의 단골 소재였습니다 1998년 김대중 정부 출범과 함께 통상 이란 두 글자는 산업부에서 외교부로 넘어갔습니다 박근혜 정부 들어 산업부의 후신인 지식경제부로 되돌려 놓으며 갈등을 빚기도 했습니다 인수위가 10년 만에 부활한 만큼 두 부처 간 물밑 다툼도 치열합니다 외교부에선 CPTPP나 쿼드 가입 등 굵직한 현안의 사령탑을 자신들이 맡아야 한다고 주장합니다 반면 산업부는 국내 산업 기반에 대한 고려 없이 국제 정치 논리로 통상을 다룰 순 없다고 맞섭니다 한미 FTA 10주년 공치사 과정에서 아전인수 식 행사를 앞다퉈 열기도 했습니다 미국의 러시아 수출 통제 대상 에서 한국이 처음에 면제받지 못한 것도 같은 맥락으로 풀이됩니다 두 부처가 통상 주도권을 두고 엇박자를 내다가 뒤늦게 수습에 나서는 촌극을 빚었다는 겁니다 이럴 바엔 미국의 무역대표부 처럼 통상 전담 조직을 대통령 직속의 제3기관으로 신설하자는 대안도 다시 거론되고 있습니다 YTN 강정규입니다 당신의 제보가 뉴스가 됩니다 YTN 검색해 채널 추가'

In [97]:
import re
# 더 좋은 방법이 있으면 추가하려고 한다. 직접 불용어들을 설정하여 추가하면 개선될 여지가 있을 것 같다.
# 숫자나 한자도 포함이 되어 있는데, 이를 제거하는 것이 나을지는 나중에 확인해보려 한다.
def cleaning(contents=None):
    for i, content in enumerate(contents):
        # 이메일제거
        pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' 
        content = re.sub(pattern=pattern, repl=' ', string=content)
        # 전화번호 제거
        pattern = '([0-9]+-[0-9]+-[0-9]*)' 
        content = re.sub(pattern=pattern, repl=' ', string=content)
        # url제거
        pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
        content = re.sub(pattern=pattern, repl=' ', string=content)
        # 한글 자음 모음 제거
        pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'
        content = re.sub(pattern=pattern, repl=' ', string=content)
        # 괄호와 괄호안 글자 제거
        pattern = r'\([^)]*\)'
        content = re.sub(pattern=pattern, repl=' ', string=content)
        pattern = r'\[[^]]*\]'
        content = re.sub(pattern=pattern, repl=' ', string=content)
        # 특수기호 제거
        pattern = '[^\w\s]'
        content = re.sub(pattern=pattern, repl=' ', string=content)
        # # 숫자 제거
        # pattern = '[0-9]+' 
        # content = re.sub(pattern=pattern, repl=' ', string=content)
        # # 숫자 포함 단어 제거
        # pattern = '[0-9]+[가-힣a-zA-Z]*' 
        # content = re.sub(pattern=pattern, repl=' ', string=content)
        # 양 끝 공백 제거
        content = content.strip()
        # 중간에 공백은 한개로 통일
        content = " ".join(content.split())

        contents[i] = content
 

In [96]:
# 전처리 함수 작동 확인
a = ['저는 content대회에서 3위를 하였습니다. "언제든" 연락주세요! [카카오톡] example [이메일] example@example.com (전화번호) 010-1111-2222']
print(a)
cleaning(a)
a

['저는 content대회에서 3위를 하였습니다. "언제든" 연락주세요! [카카오톡] example [이메일] example@example.com (전화번호) 010-1111-2222']


['저는 content대회에서 3위를 하였습니다 언제든 연락주세요 example']

In [98]:
# 초기 저장시에 index 설정을 건드리지 않아서 불러올 떄 처리를 해줘야 함
import pandas as pd
newsdata_origin = pd.read_csv('./dataset/newsdata.csv',index_col=0)
newsdata = newsdata_origin.copy()
# 위에서 사진만 있는 경우 패스하도록 해서 null로 들어가 있는 것이 확인되었는데, 2400개중 7개라서 그냥 삭제하였고 인덱스를 다시 설정함
newsdata = newsdata.dropna()
newsdata = newsdata.reset_index().drop('index',axis=1)

In [99]:
newsdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2393 entries, 0 to 2392
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   data     2393 non-null   object
 1   section  2393 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 37.5+ KB


In [100]:
newsdata.head()

,data,section
0,발언하는 권영세 인수위 부위원장(서울=연합뉴스) 권영세 인수위 부위원장이 ...,0
1,(서울=연합뉴스) 터키를 방문 중인 김부겸 국무총리가 19일(현지...,0
2,동영상 뉴스 권영세 대통령직인수위원회 부위원장이 코로나19에 확...,0
3,"경기 8만7573명, 서울 6만5970명, 인천 1만8175명 등 확진신규...",0
4,동영상 뉴스 더불어민주당 의원들을 중심으로 윤석열 당선인의 청와...,0


In [101]:
# 텍스트 전처리
cleaning(newsdata.data)
newsdata

C:\Users\chonh\AppData\Local\Temp/ipykernel_20400/1550849439.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contents[i] = content


,data,section
0,발언하는 권영세 인수위 부위원장 서울 연합뉴스 권영세 인수위 부위원장이 18일 오전...,0
1,서울 연합뉴스 터키를 방문 중인 김부겸 국무총리가 19일 현지시간 오전 앙카라 한국...,0
2,동영상 뉴스 권영세 대통령직인수위원회 부위원장이 코로나19에 확진됐습니다 인수위 관...,0
3,경기 8만7573명 서울 6만5970명 인천 1만8175명 등 확진신규확진 소폭 줄...,0
4,동영상 뉴스 더불어민주당 의원들을 중심으로 윤석열 당선인의 청와대 이전 추진과 관련...,0
...,...,...
2388,눈썹을 펴지 못하고 떠난 당신에게 박동욱 지음 궁리 372쪽 1만6800원 이미 아...,3
2389,김지우 지음 산지니독자의 취향이나 연령을 고려하지 않은 상태에서 추천도서는 있을 수...,3
2390,베스트 닥터의 베스트 건강팁 중년 이후 잇몸질환 전조 증세는 잇몸 피나거나 시리면 ...,3
2391,뉴컨피던스 이안 로버트슨 지음 임현경 옮김 알에이치코리아 392쪽 1만 6800원1...,3


In [44]:
# 훈련/테스트 데이터셋 분할 및 텍스트 토큰화, 벡터화, 모델 구성
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X = newsdata.data
y = newsdata.section
# shuffle의 경우 default는 True입니다. 섞어주었다는 것을 명시하기 위하여 적었기에 별로 의미는 없습니다.
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0,shuffle=True)

okt = Okt()
def okt_tokenizer(text):
    tokens_ko = okt.morphs(text)
    return tokens_ko

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환. 
tfidf_vect = TfidfVectorizer(tokenizer=okt_tokenizer ,ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(X_train)
tfidf_matrix_train = tfidf_vect.transform(X_train)
tfidf_matrix_test = tfidf_vect.transform(X_test)

# LogisticRegression을 이용하여 학습/예측/평가 수행. Rf의 경우 좀 더 낮아서 로지스틱회귀로 결정하였음.
lr = LogisticRegression(random_state=0)
lr.fit(tfidf_matrix_train , y_train)
pred = lr.predict(tfidf_matrix_test)
print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

C:\Users\chonh\anaconda3\envs\cakd5\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TF-IDF Logistic Regression 의 예측 정확도는 0.784


In [45]:
# 최적 C 값 도출 튜닝 수행. CV는 5 Fold셋으로 설정.
from sklearn.model_selection import GridSearchCV
params = {'C':[0.01, 0.1, 1, 3, 5]}
grid_clf = GridSearchCV(lr,param_grid=params, cv=5,scoring='accuracy',verbose=0, n_jobs=-1)
grid_clf.fit(tfidf_matrix_train, y_train)
best_clf = grid_clf.best_estimator_

print('Logistic Regression best C parameter :',grid_clf.best_params_ )
print('Logistic Regression best score : ', round(grid_clf.best_score_,3))

# 최적 C 값으로 학습된 grid_cv로 예측 수행하고 정확도 평가. 
preds = best_clf.predict(tfidf_matrix_test)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,preds)))

Logistic Regression best C parameter : {'C': 1}
Logistic Regression best score :  0.777
TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.784
